<a href="https://colab.research.google.com/github/Neiluj35/Projet_recherche/blob/main/Code_en_construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer

Modèle pas encore stabilisé !

In [ ]:
import numpy as np
import itertools
from scipy.special import erf, erfc

def calcul_proba_codebook_specifique():

    C = np.array([
        [-1,  1, -1, -1,  1, -1, -1],
        [ 1, -1, -1, -1, -1,  1, -1],
        [-1,  1, -1, -1, -1,  1, -1],
        [ 1, -1, -1,  1, -1, -1, -1],
        [-1, -1,  1, -1, -1, -1,  1],
        [-1, -1,  1, -1,  1, -1,  1],
        [-1, -1, -1,  1,  1, -1,  1],
        [ 1,  1, -1, -1,  1, -1,  1],
        [-1, -1,  1,  1,  1,  1,  1],
        [-1,  1, -1,  1, -1,  1,  1]
    ])

    K, M = C.shape  # K=10 utilisateurs, M=7 taille du code

    p = 0.01
    sigma_N = 1.0
    sigma = 1.0

    alpha_true = np.random.binomial(1, p, K)
    h=np.ones(K)
    #h = np.random.normal(0, sigma_N, K)
    noise = np.random.normal(0, sigma, M)

    # Y = Somme(h_k * alpha_k * C_k) + bruit
    Y = np.zeros(M)
    for k in range(K):
        Y += h[k] * alpha_true[k] * C[k]
    Y += noise

    print(f"Codebook : {K} utilisateurs, Longueur {M}")
    print(f"Y : {np.round(Y, 2)}")
    print("-" * 80)
    print(f"{'Utilisateur':<12} | {'Projection (y)':<15} | {'Vraisemblance L1':<18} | {'P(Alpha=1|Y)':<15} | {'Vrai Alpha'}")
    print("-" * 80)

    probas_calculees = []

    for i in range(K):
        y_proj = np.dot(Y, C[i])

        '''

        factor_A = np.sqrt(M) * np.sqrt(np.pi / 2) * sigma_N
        exponent = - (y_proj**2) / (2 * M * (M + 1) * (sigma_N**2))
        term_E = np.exp(exponent)
        term_S = np.sqrt(M / (M + 1))

        arg_erf = y_proj / (np.sqrt(2) * np.sqrt(M + 1) * sigma_N)
        term_erf = 1 + erf(arg_erf)

        arg_erfc = y_proj / (np.sqrt(2 * M) * sigma_N)
        term_erfc = erfc(arg_erfc)

        likelihood_active = factor_A * ( (term_E * term_S * term_erf) + term_erfc )
        '''
        factor = np.sqrt(M) * np.sqrt(np.pi / 2) * sigma_N
        term_erfc = erfc((0.2444*K*p-(M/2))/(sigma_N*np.sqrt(2*M)))
        likelihood_active = factor*(1 + term_erfc)

        # Si alpha=0
        sigma_proj = np.sqrt(M) * sigma
        likelihood_inactive = (1.0 / (np.sqrt(2 * np.pi) * sigma_proj)) * np.exp(- (y_proj**2) / (2 * sigma_proj**2))

        # Bayes
        numerator = likelihood_active * p
        denominator = (likelihood_active * p) + (likelihood_inactive * (1 - p))

        prob_final = numerator / denominator
        probas_calculees.append(prob_final)

        print(f"User {i+1:<7} | {y_proj:15.2f} | {likelihood_active:18.4f} | {prob_final:15.4f} | {alpha_true[i]}")

    return np.array(probas_calculees), Y, C, h

probas_calculees, Y, C, h = calcul_proba_codebook_specifique()

Codebook : 10 utilisateurs, Longueur 7
Y : [ 0.69  1.63 -1.38 -1.7  -0.06  0.38 -0.03]
--------------------------------------------------------------------------------
Utilisateur  | Projection (y)  | Vraisemblance L1   | P(Alpha=1|Y)    | Vrai Alpha
--------------------------------------------------------------------------------
User 1       |            3.62 |             9.3213 |          0.6141 | 0
User 2       |            2.61 |             9.3213 |          0.5042 | 0
User 3       |            4.50 |             9.3213 |          0.7260 | 0
User 4       |           -1.56 |             9.3213 |          0.4264 | 0
User 5       |           -2.35 |             9.3213 |          0.4811 | 0
User 6       |           -2.46 |             9.3213 |          0.4907 | 0
User 7       |           -3.11 |             9.3213 |          0.5548 | 0
User 8       |            4.93 |             9.3213 |          0.7794 | 0
User 9       |           -5.10 |             9.3213 |          0.8004 | 0
Us

Partie grover en construction selon le modèle qui sera construit


In [ ]:
import numpy as np
import itertools
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator

def detect_alpha_bar(Y, C, h):
    K, M = C.shape
    alpha_bar = np.zeros(K, dtype=int)
    for i in range(K):
        proj = np.dot(Y, C[i])
        limit = abs(M * h[i]) / 2.0

        if -limit <= proj <= limit:
            alpha_bar[i] = 0
        else:
            alpha_bar[i] = 1
    return alpha_bar

def reconstruct_Y(alpha_vec, C, h):
    """Somme(alpha_i * c_i * h_i)"""
    Y_hat = np.zeros(C.shape[1])
    for k in range(len(alpha_vec)):
        if alpha_vec[k] == 1:
            Y_hat += h[k] * C[k]
    return Y_hat

def cost_function(alpha_state, Y, C, h):
    """||Y - Y_alpha||"""
    Y_alpha = reconstruct_Y(alpha_state, C, h)
    return np.linalg.norm(Y - Y_alpha)

def get_initial_state_vector(probas_calculees):
    K = len(probas_calculees)
    states = list(itertools.product([0, 1], repeat=K))
    amplitudes = []

    for state in states:
        prob_joint = 1.0
        for i, bit in enumerate(state):
            p_i = probas_calculees[i]
            p_i = np.clip(p_i, 0.0, 1.0)
            prob_joint *= p_i if bit == 1 else (1 - p_i)

        amplitudes.append(np.sqrt(prob_joint))

    amplitudes = np.array(amplitudes)
    norm = np.linalg.norm(amplitudes)
    if norm > 0:
        amplitudes /= norm
    return amplitudes

def grover_oracle_threshold(K, Y, C, h, threshold):
    """Marque les états dont la distance est < seuil"""
    num_states = 2**K
    diag = np.ones(num_states)

    for idx, alpha in enumerate(itertools.product([0,1], repeat=K)):
        cost = cost_function(alpha, Y, C, h)
        if cost <= threshold:
            diag[idx] = -1 #Inversion de phase

    return Operator(np.diag(diag))

def get_weighted_diffuser(amplitudes_init):
    """Diffuseur 2|psi><psi| - I"""
    psi = amplitudes_init.reshape(-1, 1)
    P = psi @ psi.conj().T
    D = 2 * P - np.eye(len(amplitudes_init))
    return Operator(D)

def grover_with_threshold(probas_calculees, Y, C, h, iterations=1):
    K = len(probas_calculees)

    alpha_bar = detect_alpha_bar(Y, C, h)
    Y_hat_bar = reconstruct_Y(alpha_bar, C, h)

    threshold = np.linalg.norm(Y - Y_hat_bar) + 1e-4
    print(f"Alpha barre (Détecteur) : {alpha_bar}")
    print(f"Seuil de distance calculé : {threshold:.4f}")


    amplitudes_init = get_initial_state_vector(probas_calculees)
    qc = QuantumCircuit(K)
    qc.initialize(amplitudes_init, range(K))

    #Opérateurs
    oracle_op = grover_oracle_threshold(K, Y, C, h, threshold)
    diffuser_op = get_weighted_diffuser(amplitudes_init)

    #Boucle Grover
    for it in range(iterations):
        qc.unitary(oracle_op, range(K), label="Oracle")
        qc.unitary(diffuser_op, range(K), label="Diffuser")

    qc.measure_all()
    return qc

if __name__ == "__main__":
    if 'probas_calculees' in locals() and 'Y' in locals() and 'C' in locals():
        qc = grover_with_threshold(probas_calculees, Y, C, h, iterations=1)

        # Simulation
        sim = Aer.get_backend('aer_simulator')
        t_qc = transpile(qc, sim)
        result = sim.run(t_qc).result()
        counts = result.get_counts()

        # Affichage
        sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
        print("\nRésultats :")
        for state, count in sorted_counts:
            vec = [int(x) for x in state]
            dist = cost_function(vec, Y, C, h)
            print(f"Etat: {state} | Counts: {count} | Distance: {dist:.4f}")
    else:
        print("Veuillez d'abord exécuter le code de calcul des probabilités.")

Alpha barre (Détecteur) : [0 0 1 0 0 1 0 1 0 0]
Seuil de distance calculé : 6.0081

Résultats :
Etat: 0010000101 | Counts: 40 | Distance: 5.4705
Etat: 0010010101 | Counts: 30 | Distance: 5.4932
Etat: 0010000100 | Counts: 29 | Distance: 5.9500
Etat: 0011000101 | Counts: 25 | Distance: 5.7568
Etat: 0011000001 | Counts: 21 | Distance: 4.0656
Etat: 0010000111 | Counts: 21 | Distance: 5.5186
Etat: 0011010101 | Counts: 20 | Distance: 5.8022
Etat: 0011010001 | Counts: 20 | Distance: 4.1149
Etat: 0010000001 | Counts: 19 | Distance: 3.8603
Etat: 0011000111 | Counts: 18 | Distance: 5.9028
Etat: 0010010001 | Counts: 18 | Distance: 3.8767
Etat: 0010110001 | Counts: 17 | Distance: 4.5626
Etat: 0110000001 | Counts: 16 | Distance: 3.7163
Etat: 0010000000 | Counts: 15 | Distance: 4.2216
Etat: 0010010100 | Counts: 15 | Distance: 6.0080
Etat: 0110010101 | Counts: 15 | Distance: 5.2327
Etat: 0010100001 | Counts: 14 | Distance: 4.4877
Etat: 0010100101 | Counts: 14 | Distance: 5.7675
Etat: 0010000011 | Cou